# Towards a working minimal MD code

I have uploaded what is close to a working MD code as well as these notes.

It is inefficient and maybe well contain some errors!

You should now be close to having the knowhow to look through it and make changes / rewrite in your own style.

This notebook just contains brief notes on the final elements I have added to get a working code 

<div class="alert alert-block alert-danger">
Note the functions here won't work on their own but need to be combined with other elements we looked at previously. 
</div>

# Neighbour list

there are different ways of doing this but you want to be able to build / compute a list of the forces you need to calculate.
The idea is to come up with a list of atom pairs (for pair potentials) that we need to calcule forces for.

The most brute force is to consider all atom pairs explicitly.

Next we can identify that $f_{ij} = -f_{ji}$ so don't need to actually do both calculations!

Later on we can consider truncating the interactions to a finite range and ignoring particles that are too far away - this can allow significant speed ups but isn't essential to start with:

In [ ]:
def calcForces(model):
    energy = 0
    # remember to reset forces at each step
    model['forcestp1'][:] = 0.0
    for atomi in range(model['natoms']):
        #avoid double counting
        for atomj in range(atomi+1, natoms):
            energy += model['potential'](model, atomi, atomj)
            fi, fj = forceij(model,atomi,atomj)
            model['forcestp1'][atomi] += fi
            model['forcestp1'][atomj] += fj
    #print(model['forcestp1'])
    
    do_thermostat = False
    if do_thermostat:
        thermostat(model)
    
    return energy

# Integrator

we need to be able to integrate our equations of motion in time. We do this numerically for which there are a plethora of algorithms available. Generally, there is a trade off between accuracy of integration and efficiency. Standard for atomistic MD simulations is the velocity verlet algorithm.

We can start off with a simple explicit Euler routine:

In [ ]:
def integrate(model):
    if model['integrator'] == 'Euler':
        pot_energy = calcForces(model)
        model['atoms'] += model['vel']*dt + 0.5*model['forcestp1']/mass*dt**2
        model['vel'] += model['forcestp1']/mass*dt

    elif model['integrator'] == 'vverlet':
        model['forcest'] =  np.copy(model['forcestp1'])
        model['atoms'] += model['vel']*dt + 0.5*model['forcest']/mass*dt**2
        pot_energy = calcForces(model)
        model['vel'] += 0.5*(model['forcest'] + model['forcestp1'])/mass*dt            

    else:
        print('no known integrator! falling back on Euler')
        pot_energy = calcForces(model)
        model['atoms'] += model['vel']*dt + 0.5*model['forcestp1']/mass*dt**2
        model['vel'] += model['forcestp1']/mass*dt
            
    # calculate the Kinetic Energy
    KE = 0    
    for i in range(model['natoms']):
        KE += np.dot(model['vel'][i],model['vel'][i])

    return pot_energy, KE

## Thermostat

Our simple system evolving according to Newton's equations of motion is completely isolated from the outside world. This means it has no means of exchanging energy with its environment. Consequently, the total energy of the system should be exactly conserved thoughout the simulation. Of course, it will not be exactly in a numerical simulation but it should be to a good approximation and with many thousands of times smaller fluctuation that in the potential or kinetic energy of the system.

If we want to approximate the effect of the system being able to exchange energy with its environment we need to artificially alter the motion of the particles so that they correspond to one of the ensembles from Statistical Mechanics that are characteristic of a small system in equilibrium with the larger whole, generally defined by the temperature. 

There are a number of thermostats used in the literature - see suggested reading for further details.
Here is an implementation of the Langevin thermostat on all the degrees of freedom

In [ ]:
def thermostat(model):

    # langevin thermostat on all degrees of freedom
    gamma = 0.1
    kbT = 0.5
    sigma = np.sqrt(2*gamma*kbT)
    for atomi in range(model['natoms']):
        model['forcestp1'][atomi] += -gamma*model['vel'][atomi] + sigma*np.random.normal(0, sigma)